<a href="https://colab.research.google.com/github/Tuevu110405/scene_text_recognition/blob/feature%2Ftraining/Scene_text_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 905.3/905.3 kB 28.0 MB/s eta 0:00:00


In [ ]:
import ultralytics
import os
import xml.etree.ElementTree as ET

ultralytics.checks()


Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.7/107.7 GB disk)


In [ ]:
def extract_data_from_xml(root_dir):
    xml_path = os.path.join(root_dir, 'data.xml')
    tree = ET.parse(xml_path)
    root = tree.getroot()

    img_paths = []
    img_sizes = []
    img_labels = []
    bboxes = []

    for img in root:
        bbs_of_img = []
        labels_of_img = []

        for bbs in img.findall('taggedRectangles'):
            for bb in bbs:
                if not bb[0].text.isalnum():
                    continue

                if ' ' in bb[0].text.lower() or ' ' in bb[0].text.lower():
                    continue

                bbs_of_img.append(
                    [
                        float(bb.attrib['x']),
                        float(bb.attrib['y']),
                        float(bb.attrib['width']),
                        float(bb.attrib['height'])
                    ]
                )
                labels_of_img.append(bb[0].text.lower())

        img_path = os.path.join(root_dir, img[0].text)
        img_paths.append(img_path)
        img_sizes.append((int(img.attrib['x']), int(img.attrib['y'])))
        bboxes.append(bbs_of_img)
        img_labels.append(labels_of_img)

    return img_paths, img_sizes, img_labels, bboxes

dataset_dir = 'SceneTrainTrain'
img_paths, img_sizes, img_labels, bboxes = extract_data_from_xml(dataset_dir)

In [ ]:
#Convert into Yolo format
def convert_to_yolo_format(image_paths, image_sizes, bounding_boxes):
    yolo_data = []

    for image_path, image_size, bboxes in zip(image_paths, image_sizes, bounding_boxes):
        image_width, image_height = image_size

        yolo_labels = []

        for bbox in bboxes:
            x, y, w, h = bbox
            x_center = (x + w / 2) / image_width
            y_center = (y + h / 2) / image_height
            width = w / image_width
            height = h / image_height

            class_id = 0

            #Convert yolo format
            yolo_label = f"{class_id} {x_center} {y_center} {width} {height}"
            yolo_labels.append(yolo_label)

        yolo_data.append((image_path, yolo_labels))

    return yolo_data

#Define class labels
class_labels = ['text']

#Convert data into Yolo format
class_labels = ['text']

#Convert data into yolo format
yolo_data = convert_to_yolo_format(imgae_paths, image_sizes, bounding_boxes)





In [ ]:
def save_data(data, scr_img_dir, save_dir):
    os.makedirs(save_dir, exist_ok=True)

    #make images and labels folder
    os.makedirs(os.path.join(save_dir, 'images'), exist_ok=True)
    os.makedirs(os.path.join(save_dir, 'labels'), exist_ok=True)

    for image_path, yolo_labels in data:
        shutil.copy(
            os.path.join(scr_img_dir,image_path),os.path.join(
                save_dir, "images"
            )
        )

        image_name = os.path.basename(image_path)
        image_name = os.path.splitext(image_name)[0]

        with open(os.path.join(save_dir, 'labels', image_name + '.txt'), 'w') as f:
            for label in yolo_labels:
                f.write(f"{label}\n")

seed = 0
val_size = 0.2
test_size = 0.125
is_shuffle = True
train_data, test_data = train_test_split(yolo_data, test_size=test_size, random_state=seed, shuffle=is_shuffle)

test_data, val_data = train_test_split(test_data, test_size=test_size, random_state=seed, shuffle=is_shuffle)

save_yolo_data_dir = "datasets/yolo_data"
os.makedirs(save_yolo_data_dir, exist_ok = True)
save_train_dir = os.path.join(save_yolo_data_dir, 'train')
save_val_dir = os.path.join(save_yolo_data_dir, 'val')
save_test_dir = os.path.join(save_yolo_data_dir, 'test')

save_data(train_data, dataset_dir, save_train_dir)
save_data(val_data, dataset_dir, save_val_dir)
save_data(test_data, dataset_dir, save_test_dir)



In [ ]:
data_yaml = {
    "path": "./datasets/yolo_data",
    "train": "train/images",
    "val": "val/images",
    "test": "test/images",
    "nc" : 1,
    "names" : class_labels
}

yolo_yaml_path = os.path.join(save_yolo_data_dir, 'data.yaml')
with open(yolo_yaml_path, 'w') as f:
    yaml.dump(data_yaml, f, default_flow_style=False)

In [ ]:
from ultralytics import YOLO

model = YOLO("yolo11m.pt")

results = model.train(
    data = yolo_yaml_path,
    epochs = 100
    imgsz = 640
    cache = True
    patience= 20
    plots = True
)

In [ ]:
model_path = 'runs/detect/train/weights/best.pt'
model = YOLO(model_path)

metrics = model.val()

In [ ]:
import os
import random
import time
import xml.etree.ElementTree as ET

import cv2
import matplotlib.pyplot as plt
import numpy as np
import timm
import torch
import torch.nn as nn
import torchvision
from PIL import Image
from sklearn.model_selection import train_test_split

from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

In [ ]:
def extract_data_from_xml(root_dir):
    xml_path = os.path.join(root_dir, 'data.xml')
    tree = ET.parse(xml_path)
    root = tree.getroot()

    img_paths = []
    img_size = []
    img_labels = []
    bboxes = []

    for img in root:
        bbs_of_img = []
        labels_of_img = []

        for bbs in img.findall('taggedRectangles'):
            for bb in bbs:

                if not bb[0].text.isalnum():
                    continue

                if "e" in bb[0].text.lower() or "n" in bb[0].text.lower():
                    continue

                bbs_of_img.append(
                    [
                        float(bb.attrib['x']),
                        float(bb.attrib['y']),
                        float(bb.attrib['width']),
                        float(bb.attrib['height'])
                    ]
                )
                labels_of_img.append(bb[0].text.lower())

        img_path = os.path.join(root_dir, img[0].text)
        img_paths.append(img_path)
        img_size.append((int(img.attrib['x']), int(img.attrib['y'])))
        bboxes.append(bbs_of_img)
        img_labels.append(labels_of_img)

    return img_paths, img_size, img_labels, bboxes

dataset_dir = 'datasets/SceneTrainTrain'
img_paths, img_size, img_labels, bboxes = extract_data_from_xml(dataset_dir)



In [ ]:
def split_bounding_boxes(img_paths, img_labels, bboxes):
    os.makedirs(save_dir, exist_ok = True)

    count = 0
    labels = []

    for img_path, img_label, bbs in zip(img_paths, img_labels, bboxes):
        img = Image.open(img_path)

        for label, bb in zip(img_label, bbs):
            cropped_img = img.crop((bb[0], bb[1], bb[0] + bb[2], bb[1] + bb[3]))

            if np.mean(cropped_img) < 35 or np.mean(cropped_img) > 220:
                continue

            if cropped_img.size[0] < 10 or cropped_img.size[1] < 10 :
                continue

            filename = f"{count:06d}.jpg"
            cropped_img.save(os.path.join(save_dir, filename))

            new_img_path = os.path.join(save_dir, filename)

            label = new_img_path + "\t" + label
            labels.append(label)
            count += 1

    print(f"Created {count} images")

    with open(os.path.join(save_dir, "labels.txt"), "w") as f:
        f.write("\n".join(labels))

save_dir = "datasets/ocr_dataset"
split_bounding_boxes(img_paths, img_labels, bboxes, save_dir)


In [ ]:
root_dir = save_dir

img_paths = []
labels = []

with open(os.path.join(root_dir, "labels.txt"), "r") as f:
    for label in f:
        labels.append(label.strip().split("\t")[1])
        img_paths.append(label.strip().split("\t")[0])

print(f"Total images: {len(img_paths)}")
print(f"Total labels: {len(labels)}")

In [ ]:
#build vocabulary
letters = [char.split(".")[0].lower() for char in labels]
letters = "".join(letters)
letters = sorted(list(set(letters)))

blank_char = "-"
chars += blank_char
vocab_size = len(chars)

print(f"Vocab: {chars}")
print(f"Vocab size: {vocab_size}")



In [ ]:
char_to_idx = {char: idx + 1 for idx, char in enumerate(sorted(chars))}
idx_to_char = {index: char for char, index in char_to_idx.items()}

In [ ]:
max_label_len = max([len(label) for label in labels])

def encode(label, char_to_idx, max_label_len):
    encoded_labels = torch.tensor(
        [char_to_idx[char] for char in label], dtype = torch.int32

    )
    label_len = len(encoded_labels)
    lengths = torch.tensor(
        label_len,
        dtype = torch.int32
    )
    padded_labels = F.pad(
        encoded_labels,
        (0, max_label_len - label_len)
        value = 0
    )

    return padded_labels, lengths


In [ ]:
def decode(encoded_sequences, idx_to_char, blank_char = "-"):
    decoded_sequences  = []

    for seq in encoded_sequences:
        decoded_label = []
        prev_char = None

        for token in seq:
            if token != 0:
                char = idx_to_char[token.item()]

                if char != blank_char:
                    if char != prev_char or prev_char == blank_char:
                        decoded_label.append(char)

                prev_char = char

        decoded_sequences.append("".join(decoded_label))

    print(f"From {encoded_sequences} to {decoded_sequences}")

    return decoded_sequences

In [ ]:
data_transforms = {
    "train" : transforms.Compose(
        [
            transforms.Resize((100, 420))
            transforms.ColorJitter(
                brightness = 0.5,
                contrast = 0.5,
                saturation = 0.5,

            ),
            transforms.Grayscale(
                num_output_channels = 1
            ),
            transforms.GaussianBlur(3),
            transforms.RandomAffine(
                degree = 1,
                shear = 1
            ),
            transforms.RandomPerspective(
                distortion_scale = 0.3,
                p = 0.5
                interpolation= 3
            ),
            transforms.RandomRotation(
                degrees = 2
            ),
            transforms.ToTensor()
            transforms.Normalize((0.5,), (0.5,))

        ]
    ),
    "val" : transforms.Compose(
        [
            transforms.Resize((100, 420)),
            transforms.Grayscale(
                num_output_channels = 1
            ),
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,))
        ]
    ),

}


In [ ]:
seed = 0
val_size = 0.2
test_size = 0.1
is_shuffle = True

X_train, X_val, y_train, y_val = train_test_split(
    img_paths,
    labels,
    test_size = val_size,
    random_state = seed,
    shuffle = is_shuffle,
)

X_train, X_test, y_train, y_test = train_test_split(
    X_train,
    y_train,
    test_size = test_size,
    random_state = seed,
    shuffle = is_shuffle,
)

In [ ]:
class STRDataset(Dataset):
    def __init__(
            self,
            X,
            y,
            char_to_idx,
            max_label_len,
            transform = None

    ):
        self.transforms = transform
        self.img_paths = X
        self.labels = y
        self.char_to_idx = char_to_idx
        self.max_label_len = max_label_len
        self.label_encoder = label_encoder

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, index):
        img_path = self.img_paths[idx]
        label = self.labels[idx]
        img = Image.open(img_path).convert("RGB")

        if self.transforms:
            img = self.transforms(img)

        if self.label_encoder:
            encoded_label, label_len = self.label_encoder(
                label, self.char_to_idx, self.max_label_len
            )

        return img, encoded_label, label_len


In [ ]:
train_dataset = STRDataset(
    X_train,
    y_train,
    char_to_idx,
    max_label_len,
    label_encoder= encode,
    data_transforms['train']
)

val_dataset = STRDataset(
    X_val,
    y_val,
    char_to_idx,
    max_label_len,
    label_encoder= encode,
    data_transforms['val']
)

test_dataset = STRDataset(
    X_test,
    y_test,
    char_to_idx,
    max_label_len,
    label_encoder= encode,
    data_transforms['val']
)

train_batch_size = 64
test_batch_size = 64 * 2

train_loader = DataLoader(
    train_dataset,
    batch_size= train_batch_size,
    shuffle = True
)

val_loader = DataLoader(
    val_dataset,
    batch_size= test_batch_size,
    shuffle = False
)

test_loader = DataLoader(
    test_dataset,
    batch_size= test_batch_size,
    shuffle = False

)

In [ ]:
class CRNN(nn.Module):
    def __init__(
            self, vocab_size, hidden_size, n_layers, dropout = 0.2, unfreeze_layers = 3
    ):
        super(CRNN, self).__init__()
        backbone = timm.create_model("resnet152", in_chans = 1, pretrained = True)
        modules = list(backbone.children())[:-2]
        self.backbone = nn.Sequential(*modules)

        for parameter in self.backbone[-unfreeze_layers:].parameters():
            parameter.requires_grad = True

        self.mapSeq = nn.Sequential(
            nn.Linear(2048,512), nn.ReLU(), nn.Dropout(dropout)
        )

        self.gru = nn.GRU(
            512,
            hidden_size,
            n_layers,
            bidirectional = True,
            batch_first = True,
            dropout = dropout if n_layers > 1 else 0
        )
        self.layer_norm = nn.LayerNorm(hidden_size * 2)

        self.out = nn.Sequential(
            nn.Linear(hidden_size * 2, vocab_size),
            nn.LogSoftmax(dim = 2)
        )

        @torch.autocast(device_type = "cuda")
        def forward(self, x):
            x = self.backbone(x)
            x = x.permute(0, 3, 1, 2)
            #Flatten the feature map
            x = x.view(x.size(0), x.size(1), -1)
            x = self.mapSeq(x)
            x, _ = self.gru(x)
            x = self.layer_norm(x)
            x = self.out(x)
            x = x.permute(1, 0, 2)

            return x


In [ ]:
hidden_size = 256
n_layers = 3
dropout_prob = 0.2
unfreeze_layers = 3
device = "cuda" if torch.cuda.is_available() else "cpu"

model = CRNN(
    vocab_size,
    hidden_size,
    n_layers,
    dropout = dropout_prob,
    unfreeze_layers = unfreeze_layers
).to(device)

In [ ]:
def evaluate(model, dataloader, criterion, device):
    model.eval()
    losses = []
    with torch.no_grad():
        for inputs, labels, labels_len in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            labels_len = labels_len.to(device)

            outputs = model(inputs)
            logits_lens = torch.full(
                size = (outputs.size(1),),
                fill_value = outputs.size(0),
                dtype = torch.long
            ).to(device)

            loss = criterion(outputs, labels, logits_lens, labels_len)
            losses.append(loss.item())

    return np.mean(losses)

def fit(
        model, train_loader, val_loader, criterion, optimizer, scheduler, device, epochs
):
    train_losses = []
    val_losses = []

    for epoch in range(epochs):
        start = time.time()
        model.train()
        batch_train_losses = []
        for idx, (inputs, labels, labels_len) in enumerate(train_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            labels_len = labels_len.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            logits_lens = torch.full(
                size = (outputs.size(1),),
                fill_value = outputs.size(0),
                dtype = torch.long
            ).to(device)

            loss = criterion(outputs, labels, logits_lens, labels_len)

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 5)
            optimizer.step()

            batch_train_losses.append(loss.item())

        train_loss = np.mean(batch_train_losses)

        train_losses.append(train_loss)

        val_loss = evaluate(model, val_loader, criterion, device)
        val_losses.append(val_loss)

        print(
            f"EPOCH {epoch + 1}:\tTrain loss: {train_loss: .4f}\tVal loss : {val_loss: .4f}\t\t Time: {time.time() - start:.2f} seconds"
        )

        scheduler.step(val_loss)

    return train_losses, val_losses

In [ ]:
epochs = 100
lr = 5e-4
weight_decay = 1e-5
scheduler_step_size = epochs * 0.5

criterion = nn.CTCLoss(
    blank = char_to_idx[blank_char]

)
optimizer = torch.optim.Adam(model.parameters(), lr = lr, weight_decay = weight_decay)
scheduler = torch.optim.lr_schduler.StepLR(optimizer, step_size = scheduler_step_size, gamma = 0.1)



In [ ]:
train_losses, val_losses = fit(
    model, train_loader, val_loader, criterion, optimizer, scheduler,
    device, epochs
)

Evaluation

In [ ]:
val_loss = evaluate(model, val_loader, criterion, device)
test_loss = evaluate(model, test_loader, criterion, device)

print("Evaluation on val/test dataset")
print(f"Val loss: {val_loss:.4f}")
print(f"Test loss: {test_loss:.4f}")

In [ ]:
save_model_path = "ocr_crnn.pt"
torch.save(model.state_dict(), save_model_path)

In [ ]:
!pip install ultralytics
import ultralytics
ultralytics.checks()

import os
import numpy as np
import timm
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torchvision import models
from torchvision import transforms
from PIL import Image

from ultralytics import YOLO

In [ ]:
text_det_model_path = 'run/detect/train/weights/best.pt'
yolo = YOLO(text_det_model_path)

Text recognition

In [ ]:
class CRNN(nn.Module):
    def __init__(self, vocab_size, hidden_size, n_layers, dropout = 0.2, unfreeze_layers = 3):
        super(CRNN, self).__init__()

        backbone = timm.create_model("resnet101", in_chans = 1, pretrained = True)
        modules = list(backbone.children())[:-2]
        modules.append(nn.AdaptiveAvgPool2d(1, None))
        self.backbone = nn.Sequential(*modules)

        for parameter in self.backbone[-unfreeze_layers:].parameters():
            parameter.requires_grad = True

        self.mapSeq = nn.Sequential(
            nn.Linear(2048,512), nn.ReLU(), nn.Dropout(dropout)
        )

        self.lstm = nn.LSTM(
            512,
            hidden_size,
            n_layers,
            bidirectional = True,
            batch_first = True,
            dropout = dropout if n_layers > 1 else 0
        )
        self.layer_norm = nn.LayerNorm(hidden_size * 2)

        self.out = nn.Sequential(
            nn.Linear(hidden_size * 2, vocab_size),
            nn.LogSoftmax(dim = 2)
        )

    def forward(self, x):
        x = self.backbone(x)
        x = x.permute(0, 3,1,2)
        x.view(x.size(0), x.size(1), -1)
        x = self.mapSeq(x)
        x, _ = self.lstm(x)
        x = self.layer_norm(x)
        x = self.out(x)
        x = x.permute(1, 0, 2)

        return x

In [ ]:
chars = '0123456789abcdefghijklmnopqrstuvwxyz-'
vocab_size = len(chars)
char_to_idx = {char: idx + 1 for idx, char in enumerate(sorted(chars))}
idx_to_char = {idx: char for char, idx in char_to_idx.items()}

hidden_size = 256
n_layers = 3
dropout_prob = 0.2
unfreeze_layers = 3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_path = 'ocr_crnn_base_best.pt'

crnn_model = CRNN(
    vocab_size,
    hidden_size,
    n_layers,
    dropout = dropout_prob,
    unfreeze_layers = unfreeze_layers
).to(device)

crnn_model.load_state_dict(torch.load(model_path))


In [ ]:
def decode(encoded_sequences, idx_to_char, blank_char = '-'):
    decoded_sequences = []

    for seq in encoded_sequences:
        decoded_label = []
        for idx, token in enumerate(seq):
            if token != 0:
                char = idx_to_char[token.item()]
                if char != blank_char:
                    decoded_label.append

        decoded_sequences.append(''.join(decoded_label))

    return decoded_sequences

In [ ]:
def text_detection(img_path, text_det_model):
    text_det_results =  text_det_model(img_path, verbose = False)[0]

    bboxes = text_det_results.boxes.xyxy.tolist()
    classes = text_det_results.boxes.cls.tolist()
    names = text_det_results.names
    confs = text_det_results.boxes.conf.tolist()

    return bboxes, classes, names, confs


In [ ]:
def text_recognition(img, data_transforms, text_reg_model, idx_to_char, device):
    transformed_img = data_transforms(img)
    transformed_img = transformed_img.unsqueeze(0)
    transformed_img = transformed_img.to(device)
    text_reg_model.eval()
    with torch.no_grad():
        logits = text_reg_model(transformed_img).detach().cpu()
    text = decode(logits.permute(1, 0, 2).argmax(2), idx_to_char)

    return text


In [ ]:
def visualize_detections(img, detections):
    plt.figure(figsize = (12,8))
    plt.imshow(img)
    plt.axis('off')

    for bbox, detected_class, confidence, transcribed_text in detections:
        x1, y1, x2, y2 = bbox
        plt.gca().add_patch(plt.Rectangle((x1, y1), x2 - x1, y2 - y1, fill = False, edgecolor = 'red', linewidth = 2))
        plt.text(
            x1, y1 - 10, f"{detected_class}: {confidence:.2f}\n{transcribed_text}",
            fontsize = 9, bbox = dict(facecolor = 'red', alpha = 0.5)
        )

    plt.show()

In [ ]:
data_transforms = {
    'train' : transforms.Compose([
        transforms.Resize((100, 420)),
        transforms.ColorJitter(brightness = 0.5, contrast = 0.5, saturation = 0.5),
        transforms.Grayscale(num_output_channels = 1),
        transforms.GaussianBlur(3),
        transforms.RandomPerpective(distortion_scale = 0.2, p = 0.3,interpolation = 3),
        transforms.RandomRotation(degree = 2)
        transforms.ToTensor()
        transforms.Normalize(mean = [0.5], std = [0.5])
    ]),
    'val' : transforms.Compose([
        transforms.Resize((100, 420)),
        transforms.Grayscale(num_output_channels = 1),
        transforms.ToTensor()
        transforms.Normalize(mean = [0.5], std = [0.5])

    ])

}

In [ ]:
def predict(img_path, data_transforms, text_det_model, text_reg_model,
            idx_to_char, device):
    bboxes, classes, names, confs = text_detection(img_path, text_det_model)
    img = Image.open(img_path)

    predictions = []
    for bbox, cls, conf in zip(bboxes, classes, confs):
        x1, y1, x2, y2 = bbox
        confidence = conf
        detected_class = cls
        name = names[int(cls)]

        cropped_image = img.crop((x1, y1, x2, y2))
        text = text_recognition(cropped_image, data_transforms, text_reg_model, idx_to_char, device)

        predictions.append((bbox, name, confidence, text))

    visualize_detections(img, predictions)
    return predictions